# NTK classifier for Cora

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        n_per_class = 20,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [5]:
def run_exp(n_seeds, data_params, model_params, verbosity_params, other_params, n_seed_l=None):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    if n_seed_l is None:
        seeds = [seed for seed in range(n_seeds)]
    else:
        seeds = n_seed_l
    for seed in seeds:
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Accuracy Mean: {np.array(acc_l).mean()}")
    print(f"Accuracy Std: {np.array(acc_l).std()}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [12]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "cora_ml"
other_params["device"] = 0
seed = 2
data_params["specification"]["seed"] = seed
verbosity_params["debug_lvl"] = "warning"
run(data_params, model_params, verbosity_params, other_params, seed)

{'accuracy': 0.8063380122184753,
 'min_ypred': -3.2557194232940674,
 'max_ypred': 2.2900896072387695,
 'min_ntklabeled': 3.180507332309975,
 'max_ntklabeled': 136.08748483150904,
 'min_ntkunlabeled': 2.712679896734373,
 'max_ntkunlabeled': 36.83677792706871,
 'cond': 26427.684045753333}

In [7]:
rng = np.random.Generator(np.random.PCG64(10))
y = rng.integers(0,2,size=10)
print(y)

[1 1 0 0 1 1 1 0 1 1]


# SVM

In [35]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "cora_ml"
model_params["pred_method"] = "svm"
n_seed_l = None
verbosity_params["debug_lvl"] = "warning"
n_seeds = 10
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params, n_seed_l)

Accuracy: ['0.86', '0.87', '0.85', '0.86', '0.78', '0.79', '0.85', '0.84', '0.82', '0.82']
Accuracy Mean: 0.8334507048130035
Accuracy Std: 0.02933545175833569
Min y_pred: ['-2.01', '-1.80', '-1.93', '-1.87', '-2.13', '-2.22', '-1.89', '-2.01', '-1.89', '-1.98']
Max y_pred: ['2.04', '1.75', '2.08', '2.14', '2.47', '2.29', '2.01', '1.82', '2.00', '2.06']
Min NTK_labeled: ['3.52', '3.30', '2.93', '3.29', '2.95', '3.51', '3.47', '1.89', '2.73', '2.61']
Max NTK_labeled: ['142.09', '168.08', '132.09', '110.09', '72.59', '108.09', '104.09', '324.07', '74.09', '139.80']
Min NTK_unlabeled: ['2.64', '3.17', '2.63', '2.63', '3.35', '2.63', '3.07', '3.23', '2.33', '3.04']
Max NTK_unlabeled: ['47.54', '39.93', '38.96', '41.18', '47.54', '47.08', '33.10', '74.87', '39.56', '43.30']
Condition: ['26316', '13169', '25396', '25245', '25365', '16955', '16290', '13427', '15812', '27280']


In [33]:
model_params["regularizer"] = 0.1
data_params["dataset"] = "pubmed"
model_params["pred_method"] = "svm"
n_seed_l = None
n_seeds = 10
other_params["device"] = "cpu"
verbosity_params["debug_lvl"] = "warning"
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params, n_seed_l=n_seed_l)

Accuracy: ['0.78', '0.76', '0.65', '0.74', '0.72', '0.66', '0.77', '0.73', '0.72', '0.77']
Accuracy Mean: 0.731424230337143
Accuracy Std: 0.04095614945257118
Min y_pred: ['-1.02', '-1.02', '-1.01', '-1.02', '-1.02', '-1.02', '-1.01', '-1.03', '-1.02', '-1.01']
Max y_pred: ['-0.90', '-0.92', '-0.90', '-0.91', '-0.90', '-0.86', '-0.92', '-0.85', '-0.85', '-0.89']
Min NTK_labeled: ['0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02']
Max NTK_labeled: ['0.50', '1.03', '0.66', '0.40', '0.96', '1.13', '0.62', '0.83', '0.59', '0.82']
Min NTK_unlabeled: ['0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02']
Max NTK_unlabeled: ['0.46', '0.33', '0.27', '0.36', '0.27', '0.73', '0.25', '0.31', '0.31', '0.33']
Condition: ['40', '47', '40', '40', '41', '44', '40', '41', '41', '43']


In [37]:
model_params["regularizer"] = 1
data_params["dataset"] = "cora"
model_params["pred_method"] = "krr"
n_seeds = 2
run_exp(n_seeds, data_params, model_params, verbosity_params, other_params)

/ceph/ssd/staff/gosl/.conda/envs/py311_ntk/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)
/ceph/ssd/staff/gosl/src/ntk-robust/exp_ntk.py:162: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=other_params["dtype"], device=device)
/ceph/ssd/staff/gosl/src/ntk-robust/exp_ntk.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requ

number of samples
 - labeled: 140 
 - val: 140 
 - test: 273 
 - unlabeled: 2155
number of samples
 - labeled: 140 
 - val: 140 
 - test: 273 
 - unlabeled: 2155
Accuracy: ['0.86', '0.84']
Min y_pred: ['-0.61', '-0.68']
Max y_pred: ['1.54', '2.93']
Min NTK_labeled: ['0.55', '0.78']
Max NTK_labeled: ['41.00', '53.00']
Min NTK_unlabeled: ['0.52', '0.64']
Max NTK_unlabeled: ['28.50', '28.00']
Condition: ['931', '939']
